In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName("mylogreg").getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/04/19 04:23:57 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
from pyspark.ml.classification import LogisticRegression

In [4]:
my_data = spark.read.format("libsvm").load("sample_libsvm_data.txt")

22/04/19 04:26:32 WARN LibSVMFileFormat: 'numFeatures' option not specified, determining the number of features by going though the input. If you know the number in advance, please specify it via 'numFeatures' option to avoid the extra scan.


In [6]:
my_data.show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(692,[127,128,129...|
|  1.0|(692,[158,159,160...|
|  1.0|(692,[124,125,126...|
|  1.0|(692,[152,153,154...|
|  1.0|(692,[151,152,153...|
|  0.0|(692,[129,130,131...|
|  1.0|(692,[158,159,160...|
|  1.0|(692,[99,100,101,...|
|  0.0|(692,[154,155,156...|
|  0.0|(692,[127,128,129...|
|  1.0|(692,[154,155,156...|
|  0.0|(692,[153,154,155...|
|  0.0|(692,[151,152,153...|
|  1.0|(692,[129,130,131...|
|  0.0|(692,[154,155,156...|
|  1.0|(692,[150,151,152...|
|  0.0|(692,[124,125,126...|
|  0.0|(692,[152,153,154...|
|  1.0|(692,[97,98,99,12...|
|  1.0|(692,[124,125,126...|
+-----+--------------------+
only showing top 20 rows



In [7]:
my_log_reg_model = LogisticRegression()

In [8]:
fitted_logreg = my_log_reg_model.fit(my_data)

22/04/19 04:29:54 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
22/04/19 04:29:54 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.ForeignLinkerBLAS
22/04/19 04:29:54 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
22/04/19 04:29:54 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS


In [9]:
log_summary = fitted_logreg.summary

In [10]:
log_summary.predictions

/home/ubuntu/.local/lib/python3.8/site-packages/pyspark/sql/context.py:125: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


DataFrame[label: double, features: vector, rawPrediction: vector, probability: vector, prediction: double]

In [11]:
log_summary.predictions.printSchema()

root
 |-- label: double (nullable = true)
 |-- features: vector (nullable = true)
 |-- rawPrediction: vector (nullable = true)
 |-- probability: vector (nullable = true)
 |-- prediction: double (nullable = false)



In [12]:
log_summary.predictions.show()

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(692,[127,128,129...|[20.3777627514872...|[0.99999999858729...|       0.0|
|  1.0|(692,[158,159,160...|[-21.114014198868...|[6.76550380000472...|       1.0|
|  1.0|(692,[124,125,126...|[-23.743613234676...|[4.87842678716177...|       1.0|
|  1.0|(692,[152,153,154...|[-19.192574012720...|[4.62137287298144...|       1.0|
|  1.0|(692,[151,152,153...|[-20.125398874699...|[1.81823629113068...|       1.0|
|  0.0|(692,[129,130,131...|[20.4890549504196...|[0.99999999873608...|       0.0|
|  1.0|(692,[158,159,160...|[-21.082940212814...|[6.97903542823766...|       1.0|
|  1.0|(692,[99,100,101,...|[-19.622713503550...|[3.00582577446132...|       1.0|
|  0.0|(692,[154,155,156...|[21.1594863606582...|[0.99999999935352...|       0.0|
|  0.0|(692,[127

In [14]:
lr_train, lr_test = my_data.randomSplit([0.7, 0.3])

In [15]:
final_model = LogisticRegression()

In [18]:
fit_final = final_model.fit(lr_train)

In [19]:
prediction_and_labels = fit_final.evaluate(lr_test)

In [20]:
prediction_and_labels.predictions.show()

/home/ubuntu/.local/lib/python3.8/site-packages/pyspark/sql/context.py:125: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(692,[122,123,124...|[16.6409904991396...|[0.99999994071980...|       0.0|
|  0.0|(692,[123,124,125...|[35.3308662367293...|[0.99999999999999...|       0.0|
|  0.0|(692,[124,125,126...|[30.0835588874436...|[0.99999999999991...|       0.0|
|  0.0|(692,[124,125,126...|[21.9826765999883...|[0.99999999971617...|       0.0|
|  0.0|(692,[126,127,128...|[21.4135828435002...|[0.99999999949858...|       0.0|
|  0.0|(692,[126,127,128...|[23.5874090778168...|[0.99999999994296...|       0.0|
|  0.0|(692,[126,127,128...|[26.0053977356139...|[0.99999999999491...|       0.0|
|  0.0|(692,[127,128,129...|[25.5065193313300...|[0.99999999999163...|       0.0|
|  0.0|(692,[128,129,130...|[21.2784908487640...|[0.99999999942605...|       0.0|
|  0.0|(692,[151

In [22]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator

In [23]:
my_eval = BinaryClassificationEvaluator()

In [26]:
my_final_roc = my_eval.evaluate(prediction_and_labels.predictions)

In [27]:
my_final_roc

1.0